In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.2 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64"

In [4]:
#The following snippet load Bert and test the model on a toy example


from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Automatically loads the correct tokenizer for the model you're using,
# AutoModelForTokenClassification: Loads a pretrained model specifically for token classification tasks like NER.
# pipeline: Hugging Face's high-level API that wraps models for easy inference. Here we’ll use it for NER


model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english" # A pretrained BERT model fine-tuned for Named Entity Recognition (NER) on the CoNLL-2003 dataset.

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)


ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

# Example
text = "Hugging Face Inc. is a company based in New York City. Its founders include Julien Chaumond and Thomas Wolf."

ner_results = ner_pipeline(text)


for entity in ner_results:
    print(entity)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'entity_group': 'ORG', 'score': 0.9919034, 'word': 'Hugging Face Inc', 'start': 0, 'end': 16}
{'entity_group': 'LOC', 'score': 0.999103, 'word': 'New York City', 'start': 40, 'end': 53}
{'entity_group': 'PER', 'score': 0.98748785, 'word': 'Julien Chaumond', 'start': 76, 'end': 91}
{'entity_group': 'PER', 'score': 0.9993845, 'word': 'Thomas Wolf', 'start': 96, 'end': 107}


In [8]:



model_checkpoint = "d4data/biomedical-ner-all" 

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)


ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

# Example
text = """
	CASE: A 28-year-old previously healthy man presented with a 6-week history of palpitations. 
      The symptoms occurred during rest, 2–3 times per week, lasted up to 30 minutes at a time 
      and were associated with dyspnea. Except for a grade 2/6 holosystolic tricuspid regurgitation 
      murmur (best heard at the left sternal border with inspiratory accentuation), physical 
      examination yielded unremarkable findings.
      """


ner_results = ner_pipeline(text)


for entity in ner_results:
    print(entity)

Device set to use cpu


{'entity_group': 'Age', 'score': 0.99710876, 'word': '28 - year - old', 'start': 10, 'end': 21}
{'entity_group': 'History', 'score': 0.99868, 'word': 'previously healthy', 'start': 22, 'end': 40}
{'entity_group': 'Sex', 'score': 0.99977785, 'word': 'man', 'start': 41, 'end': 44}
{'entity_group': 'Clinical_event', 'score': 0.9994759, 'word': 'presented', 'start': 45, 'end': 54}
{'entity_group': 'Duration', 'score': 0.9992834, 'word': '6 - week', 'start': 62, 'end': 68}
{'entity_group': 'Sign_symptom', 'score': 0.9999378, 'word': 'pal', 'start': 80, 'end': 83}
{'entity_group': 'Sign_symptom', 'score': 0.98360693, 'word': '##pit', 'start': 83, 'end': 86}
{'entity_group': 'Sign_symptom', 'score': 0.95622605, 'word': 'symptoms', 'start': 105, 'end': 113}
{'entity_group': 'Frequency', 'score': 0.6091099, 'word': '2', 'start': 136, 'end': 137}
{'entity_group': 'Detailed_description', 'score': 0.88969606, 'word': '– 3 times', 'start': 137, 'end': 145}
{'entity_group': 'Detailed_description', '

# in NER output because with BERT tokenizer, which uses WordPiece tokenization, words like "ho", "##los", and "##ystolic" get splitted. This means it splits unknown or rare words into subword units — especially in biomedical or clinical texts, even if
# group entities is set equal to True. A custom function can be used to merge subwords in postprocessing

# BERT tokenizer uses WordPiece tokenization, so in NER output, words like "ho", "##los", and "##ystolic" get splitted. This means model splits unknown or rare words into subword units — especially in biomedical or clinical texts, even if
# group entities is set equal to True. A custom function can be used to merge subwords in postprocessing